In [1]:
import numpy as np
import pandas as pd
import glob
import os

import matplotlib.pyplot as plt
from numba import njit , prange
from matplotlib.colors import TwoSlopeNorm
import progressbar
import hints
from itertools import permutations
import matplotlib.gridspec as gridspec
import matplotlib
from matplotlib.ticker import AutoMinorLocator
from scipy.io import loadmat
import glob
import imageio.v2 as imageio
import scipy

In [2]:
from jitcsde import jitcsde, y, t


In [9]:
def KMC_ABC(x,dt,N,M,mode,method):
    
    # if method=='abolfazl':
    #     x, y = caly(x, N)
    #     Dcoeffs, ind, col = calD1coef(x, y, dt, N, M, mode)
    # elif method=='hints':
    #     calulator = hints.kmcc(ts_array=np.swapaxes(x,0,1), dt=dt, interaction_order=M, estimation_mode='drift')
    #     Dcoeffs = calulator.get_coefficients()  
    # elif method=='hints2' or method=='amin':   
    #     Dcoeffs = kmc(np.swapaxes(x,0,1), dt=dt, order=[1,2,3], mode='drift', **{'window_order':5, 'solver':'both', 'threshold':1e-15})   # 0
    # else:
    #     raise ValueError("Method options are: 'abolfazl', 'hints', 'hints2' ")

    calulator = hints.kmcc(ts_array=np.swapaxes(x,0,1), dt=dt, interaction_order=M, estimation_mode='drift')
    Dcoeffs = calulator.get_coefficients()
    Dcoeffs = np.swapaxes(Dcoeffs.to_numpy(), 0, 1)

    Constant = np.zeros(N)
    A = np.zeros((N, N))        
    B = np.zeros((N, N, N))    
    C = np.zeros((N, N, N, N))  

    
    num_first_order = N
    num_second_order = int(N * (N + 1) / 2)
    num_third_order = int(N * (N + 1) * (N + 2) / 6)

    
    if 0 in M:  
        Constant[:] = Dcoeffs[:, 0]
        A[:, :] = Dcoeffs[:, 1:N+1]  
    else:
        A[:, :] = Dcoeffs[:, :N]     
    
    
    if 2 in M:  
        if 0 in M:
            counter = num_first_order + 1
        else:    
            counter = num_first_order
        for i in range(N):
            for j in range(i, N):
                for idx in set(permutations((i, j))): B[:, idx[0], idx[1]] = Dcoeffs[:, counter]
                counter += 1

    if 3 in M:  
        if 0 in M:
            counter = num_first_order + num_second_order + 1
        else:
            counter = num_first_order + num_second_order
        for i in range(N):
            for j in range(i, N):
                for k in range(j, N):
                    for idx in set(permutations((i, j, k))): C[:, idx[0], idx[1], idx[2]] = Dcoeffs[:, counter]
                    counter += 1

    return Constant, A, B, C, Dcoeffs

def KMC_ABC_overtime(data,win,ol,N,M,mode,dt,tstart,zmean,uvar,method):
    dx=win*(100-ol)/100
    n_win=int((data.shape[1]-win)/dx+1)
    alleval=[]
    allevec=[]
    allcons=np.zeros((n_win,N))
    alla=np.zeros((n_win,N,N))
    allb=np.zeros((n_win,N,N,N))
    allc=np.zeros((n_win,N,N,N,N))
    alldcoefllist=[]
    t=int(win/2+tstart)
    tlist=[]
    ii=0
    tarray=np.arange(int(win / 2 + tstart), int(data.shape[1] - win / 2), dx)

    # bar = progressbar.ProgressBar(maxval=len(tarray),  redirect_stdout=True,  redirect_stderr=True, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    # bar.start()


    for t in tarray:
        tlist.append(t)
        start=int(t-win/2)
        end=int(t+win/2)
        x=np.zeros((N,win))
        x[:,:]=data[:,start:end]

        for i in range(N):
            if zmean==True:
                x[i,:]=x[i,:]-np.mean(x[i,:])
            if uvar==True:
                x[i,:]=x[i,:]/np.std(x[i,:])

        cons, A, B, C, Dcoeffs=KMC_ABC(x,dt,N,M,mode,method)
        allcons[ii,:]=cons[:]
        alla[ii,:,:]=A[:,:]
        allb[ii,:,:,:]=B[:,:,:]
        allc[ii,:,:,:,:]=C[:,:,:,:]
        alldcoefllist.append(Dcoeffs)
        
        w, v = np.linalg.eig(A)
        alleval.append(w)
        allevec.append(v)
        ii+=1
    #     bar.update(ii)
    # bar.finish()

    alldcoefllist=np.array(alldcoefllist)
    alleval=np.array(alleval)
    allevec=np.array(allevec)


    return tlist, allcons, alla, allb, allc, alldcoefllist, alleval, allevec

In [18]:
data_1 = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/3D Limit Cycle Data & Coef/VanDerPol3D_noise0.1_dt0.005_mu1_alpha0.1_beta0.5.npy")
data_1.shape


(3, 5000001)

In [12]:
dt=0.005
N=3
M=[0, 1, 2, 3]
mode='list'
win=50000
ol=0
tstart=0
zmean=False
uvar=False
method='hints'
tlist, allcons_pol1, alla_pol1, allb_pol1, allc_pol1, alldcoefllist, alleval, allevec=KMC_ABC_overtime(data_1,win,ol,N,M,mode,dt,tstart,zmean,uvar,method)
np.save(f"VanDerPol3D_Pol1_A_win={win}_overlap={ol}_mean={zmean}_unitvar={uvar}.npy", alla_pol1)
np.save(f"VanDerPol3D_Pol1_B_win={win}_overlap={ol}_mean={zmean}_unitvar={uvar}.npy", allb_pol1)
np.save(f"VanDerPol3D_Pol1_C_win={win}_overlap={ol}_mean={zmean}_unitvar={uvar}.npy", allc_pol1)
np.save(f"VanDerPol3D_Pol1_Const_win={win}_overlap={ol}_mean={zmean}_unitvar={uvar}.npy", allcons_pol1)

In [4]:
A_pol1_all = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/3D Limit Cycle Data & Coef/VanDerPol3D_Pol1_A_win=50000_overlap=0_mean=False_unitvar=False.npy")
B_pol1_all = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/3D Limit Cycle Data & Coef/VanDerPol3D_Pol1_B_win=50000_overlap=0_mean=False_unitvar=False.npy")
C_pol1_all = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/3D Limit Cycle Data & Coef/VanDerPol3D_Pol1_C_win=50000_overlap=0_mean=False_unitvar=False.npy")

Const_pol1_all = np.load("C:/Users/alish/Desktop/exploring stability project/Power Grids/Power Grid Abolfazl/Power Grid/3D Limit Cycle Data & Coef/VanDerPol3D_Pol1_Const_win=50000_overlap=0_mean=False_unitvar=False.npy")


In [18]:
win_num = len(Const_pol1_all)
print(win_num)


100


In [35]:
from scipy import stats

def MeanWithCutoff_Apply(win_los, M):
    win_num = len(M)
    M_new = np.copy(M[win_los:win_num])
    M_mean = np.mean(M_new, axis=0)
    M_std = np.std(M_new, axis=0, ddof=1)
    M_cut = np.copy(M_mean)
    M_cut[(abs(M_mean) < stats.t.ppf(0.995, win_num-win_los)*abs(M_std/np.sqrt(win_num-win_los)))] = 0
    return M_cut

def MedianWithCutoff_Apply(win_los, M):
    win_num = len(M)
    M_new = np.copy(M[win_los:win_num])
    M_median = np.median(M_new, axis=0)
    M_std = np.std(M_new, axis=0, ddof=1)
    M_cut = np.copy(M_median)
    M_cut[(abs(M_median) < stats.t.ppf(0.995, win_num-win_los)*abs(M_std/np.sqrt(win_num-win_los)))] = 0
    return M_cut


In [31]:
from jitcsde import jitcsde, y, t
def jit_equation_0_3(Const, A, B, C):
    n = A.shape[0]
    eq = []
    for i in range(n):
        eq.append(Const[i] + sum(A[i,j] * y(j) for j in range(n)) + sum(B[i,j,k] * y(j) * y(k) for j in range(n) for k in range(j,n)) + sum(C[i,j,k,l] * y(j) * y(k) * y(l) for j in range(n) for k in range(j,n) for l in range(j,k,n)))
    return eq



In [33]:
jit_equation_0_3(MeanWithCutoff_Apply(2, Const_pol1_all )  , MeanWithCutoff_Apply(2, A_pol1_all), MeanWithCutoff_Apply(2, B_pol1_all), MeanWithCutoff_Apply(2, C_pol1_all))

[-0.00319190381187527*y(0)**2*y(2) + 0.00923229048701736*y(0) + 0.992553776158531*y(1) + 0.0296052977837912*y(2),
 -0.00495228795753415 - 0.0011487624483629*y(0)*y(1) - 0.998839498895175*y(0)**2*y(1) + 0.0104999748812881*y(0)**2*y(2) + 0.000878736597937811*y(0)**2 - 0.991326488254312*y(0) + 1.03188412711359*y(1) + 0.939161929625077*y(2),
 0.00543115755012909 - 0.000917400599954498*y(0)*y(1) + 0.00246155446189248*y(0)*y(2) - 0.50328932147701*y(0) - 0.107205835491253*y(2)]

In [34]:
jit_equation_0_3(MedianWithCutoff_Apply(2, Const_pol1_all )  , MedianWithCutoff_Apply(2, A_pol1_all), MedianWithCutoff_Apply(2, B_pol1_all), MedianWithCutoff_Apply(2, C_pol1_all))

[-0.00123634821914874*y(0)*y(1) - 0.00159538149647737*y(0)**2*y(1) - 0.00554247452067*y(0)**2*y(2) + 0.992543769658547*y(1) + 0.0390127437906627*y(2),
 -0.000966846500266367*y(0)*y(1) + 0.00129822070015778*y(0)*y(2) - 0.998430345943723*y(0)**2*y(1) + 0.00927980218206652*y(0)**2*y(2) - 0.00694317879882183*y(2)*y(1)**2 + 0.00326011619002192*y(2)**2 - 0.989701912572132*y(0) + 1.03408718913802*y(1) + 0.937584058830492*y(2),
 0.00397723718367502*y(0)*y(2) - 0.00527758800403302*y(2)*y(1)**2 - 0.50041578240764*y(0) - 0.0974447031880908*y(2)]

In [36]:
(1 < 2)

True

In [5]:
folder_path = os.getcwd()

In [17]:
pd.read_csv(f'{folder_path}\\Amin-Real-9D.csv');

In [28]:
def KMC_df_overtime(data,win,ol,N,M,estimation_mode,dt,tstart,zmean,uvar):
    dx=win*(100-ol)/100
    # n_win=int((data.shape[1]-win)/dx+1)
    alldcoefllist=[]
    t=int(win/2+tstart)
    tlist=[]
    # ii=0
    tarray=np.arange(int(win / 2 + tstart), int(data.shape[1] - win / 2), dx)

    # bar = progressbar.ProgressBar(maxval=len(tarray),  redirect_stdout=True,  redirect_stderr=True, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    # bar.start()


    for t in tarray:
        tlist.append(t)
        start=int(t-win/2)
        end=int(t+win/2)
        x=np.zeros((N,win))
        x[:,:]=data[:,start:end]

        for i in range(N):
            if zmean==True:
                x[i,:]=x[i,:]-np.mean(x[i,:])
            if uvar==True:
                x[i,:]=x[i,:]/np.std(x[i,:])
        

        calulator = hints.kmcc(ts_array=np.swapaxes(x,0,1),
                                dt=dt, interaction_order=M, estimation_mode=estimation_mode)
        Dcoeffs = calulator.get_coefficients()

        alldcoefllist.append(Dcoeffs)
        # ii+=1
    #     bar.update(ii)
    # bar.finish()

    # alldcoefllist=np.array(alldcoefllist)
    # alleval=np.array(alleval)
    # allevec=np.array(allevec)


    return tlist, alldcoefllist

In [29]:
dt=0.005
M=[0, 1, 2, 3]
N=3
estimation_mode='drift'
win=50000
ol=0
tstart=0
zmean=False
uvar=False
tlist, alldcoefllist = KMC_df_overtime(data_1,win,ol,N,M,estimation_mode,dt,tstart,zmean,uvar)

In [42]:
mean_coefs = np.mean(np.array(alldcoefllist), axis=0)

In [50]:
(alldcoefllist[0].index)

Index(['1', 'x1', 'x2', 'x3', 'x1x1', 'x1x2', 'x1x3', 'x2x2', 'x2x3', 'x3x3',
       'x1x1x1', 'x1x1x2', 'x1x1x3', 'x1x2x2', 'x1x2x3', 'x1x3x3', 'x2x2x2',
       'x2x2x3', 'x2x3x3', 'x3x3x3'],
      dtype='object')

In [43]:
mean_coefs_df = pd.DataFrame(mean_coefs, index=alldcoefllist[0].index, columns=alldcoefllist[0].columns)

In [47]:
print("Mean DataFrame shape:", mean_coefs_df.shape)
print("\nFirst few rows of mean DataFrame:")
print(mean_coefs_df)

Mean DataFrame shape: (20, 3)

First few rows of mean DataFrame:
               0         1         2
1       0.000846 -0.004839  0.004952
x1      0.009121 -0.990263 -0.503267
x2      0.993092  1.030073 -0.001146
x3      0.028087  0.942990 -0.104228
x1x1   -0.000403  0.000836 -0.000672
x1x2   -0.000419 -0.001154 -0.000759
x1x3    0.000342  0.000200  0.002211
x2x2    0.000104  0.000787 -0.000022
x2x3   -0.000537 -0.000774 -0.001224
x3x3    0.000580  0.000836  0.000659
x1x1x1 -0.002053 -0.000044  0.000649
x1x1x2 -0.001179 -0.998551  0.001088
x1x1x3 -0.003027  0.009923 -0.000383
x1x2x2 -0.000630 -0.011247 -0.000621
x1x2x3 -0.000138  0.012838 -0.001869
x1x3x3  0.000703 -0.004451  0.002417
x2x2x2  0.000232  0.000168  0.000577
x2x2x3  0.001558 -0.005989 -0.002105
x2x3x3 -0.006082  0.009731  0.005447
x3x3x3  0.001054  0.002542 -0.004786
